In [13]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

replacement_value = 'dummy_repalcement_value'
path = '/home/td/Documents'

In [46]:
df = pd.read_csv('{path}/tickets.csv'.format(path=path), nrows = 10000)

df_labeled = df.dropna(subset = ['Make'])
df_labeled = df_labeled.sample(frac = 1, random_state = 1)
df_analysis = df_labeled[:int(df_labeled.shape[0] * .75)]
df_holdout = df_labeled[int(df_labeled.shape[0] * .75):]

In [47]:
class OHE():
    
    def __init__(self, training_series, min_perc = .01, col_name = '', nan_treatment = 'mode'):
        if nan_treatment == 'mode':
            training_series = training_series.fillna(training_series.mode())
        training_series = training_series.astype(str)
        
        self.sk_ohe = OneHotEncoder(handle_unknown = 'ignore')
        self.valid_values = [i for  i, j in dict(training_series.value_counts()).items() if j >= min_perc]
        training_values_to_replace = [i for i in training_series.unique() if i not in self.valid_values]
        training_series = training_series.replace(training_values_to_replace, [replacement_value for _ in training_values_to_replace])
        training_np = training_series.values.reshape(-1, 1)
        self.sk_ohe.fit(training_np)
        self.col_names = ['{col_base_name}_{value_name}'.format(col_base_name=col_name, value_name = i) for i in self.sk_ohe.categories_] 
        
    def transform(self, prediction_series):
        prediction_values_to_replace = [i for i in prediction_series.unique() if i not in self.valid_values]
        prediction_series = prediction_series.replace(prediction_values_to_replace, [replacement_value for _ in prediction_values_to_replace])
        prediction_np = prediction_series.values.reshape(-1, 1)
        output = self.sk_ohe.transform(prediction_np)
        output_df = pd.DataFrame(data = output,
                                columns = self.col_names)
        return output_df
        


In [54]:
class DataManager():
    
    def __init__(self, df_train, df_val, problem_setup = 'full_group'):
        self.df_train = df_train
        self.df_val = df_val
        self.problem_setup = problem_setup
        self.model = RandomForestClassifier()
        self.fit_data_pipeline()
        self.evaluate()
        
    def fit_data_pipeline(self):
        self.rp_state_plate_ohe = OHE(self.df_train['RP State Plate'].copy())
        self.color_ohe = OHE(self.df_train['Color'].copy())
        self.agency_ohe = OHE(self.df_train['Agency'].copy())
        self.route_ohe = OHE(self.df_train['Route'].copy())
        self.violation_code_ohe = OHE(self.df_train['Violation code'].copy())
        self.violation_code_ohe = OHE(self.df_train['Violation Description'].copy())
        self.numeric_cols = ['Fine amount']
        
        #TODO: add location features
        train_dfs = [self.rp_state_plate_ohe.transform(self.df_train['RP State Plate'].copy()),
                    self.rp_state_plate_ohe.transform(self.df_train['Color'].copy()),
                    self.rp_state_plate_ohe.transform(self.df_train['Agency'].copy()),
                    self.rp_state_plate_ohe.transform(self.df_train['Route'].copy()),
                    self.rp_state_plate_ohe.transform(self.df_train['Violation code'].copy()),
                    self.rp_state_plate_ohe.transform(self.df_train['Violation Description'].copy())]
        
        numeric_df = self.df_train[self.numeric_cols].reset_index(drop = True)
        train_dfs.append(numeric_df)
        train_data = pd.concat(train_dfs)
        train_labels = self.process_label(self.df_train)
        print(pd.isnull(train_data))
        self.model.fit(train_data, train_labels)
        
    def evaluate():
        val_data = self.process_features(self.df_val)
        val_label = self.process_features(self.df_train) 
        preds = self.model.predict(val_data)
        print(f1_score(preds, val_label))
        
    def process_label(self, df):
        if self.problem_setup == 'full_group':
            top_makes = df['Make'].value_counts()[:25].index.tolist()
            y = self.df_train['Make'].isin(top_makes).astype(int)
            return y
    
    def process_features(self, df):
        dfs = [self.rp_state_plate_ohe.transform(df['RP State Plate'].copy()),
                    self.rp_state_plate_ohe.transform(df['Color'].copy()),
                    self.rp_state_plate_ohe.transform(df['Agency'].copy()),
                    self.rp_state_plate_ohe.transform(df['Route'].copy()),
                    self.rp_state_plate_ohe.transform(df['Violation code'].copy()),
                    self.rp_state_plate_ohe.transform(df['Violation Description'].copy())]
        
        numeric_df = df[self.numeric_cols].reset_index(drop = True)
        dfs.append(numeric_df)
        data = pd.concat(train_dfs)
        return data
    
    

In [55]:
dm = DataManager(df_analysis, df_holdout)

        

      Fine amount  \
0            True   
1            True   
2            True   
3            True   
4            True   
5            True   
6            True   
7            True   
8            True   
9            True   
10           True   
11           True   
12           True   
13           True   
14           True   
15           True   
16           True   
17           True   
18           True   
19           True   
20           True   
21           True   
22           True   
23           True   
24           True   
25           True   
26           True   
27           True   
28           True   
29           True   
...           ...   
3704        False   
3705        False   
3706        False   
3707        False   
3708        False   
3709        False   
3710        False   
3711        False   
3712        False   
3713        False   
3714        False   
3715        False   
3716        False   
3717        False   
3718        False   
3719        F

/home/td/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/td/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: setting an array element with a sequence.